# **Melanoma Detection** - Training was performed in a kaggle notebook

In [ ]:
!pip install --upgrade efficientnet-pytorch
!pip install pretrainedmodels
!pip install wtfml

## Create new ground truth file for new image data

In [ ]:
import os
import glob
import pandas as pd

path = r"../input/isic-2019-test-input-512/ISIC_2019_Test_Input_512" # Path to the folder where we store all images for which we want to create the new ground truth file!

csv_df_export_path = r"./test_gt_empty_sorted.csv" # Output path of the csv file


#############################################################################################################


image_names = []

for i,file_name in enumerate(glob.glob(os.path.join(path, "*jpg"))):
    basename = os.path.basename(file_name)[:-4]
    image_names.append(basename)    
    
tryout_df = pd.DataFrame(columns=['MEL', "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC", "UNK"])

tryout_df.insert(loc=0, column="image", value=image_names)

tryout_df = tryout_df.fillna(0)

tryout_df_sorted = tryout_df.sort_values("image")

tryout_df_sorted.to_csv(csv_df_export_path, index=False)

## Hyperparameter

In [ ]:

"""You need to split the image data (which you downloaded from the ISIIC website) according to the gt_train and gt_val csv files into two folders
The paths to these folders need to be put into the variables input_path_training_img and input_path_val_img. Spliting the image data can be done 
with val_split.py

Additionally, there is also a resize.py file in the gitlab repository, which resizes your images to the wanted size.

If you have question regarding this CNN script, the val_split.py file or the resize.py file, just text me.

""" 


input_path_training_img = "../input/image512/Train_Data_2019_512/Train_Data_2019_512"
input_path_val_img = "../input/image512/Val_Data_2019_512/Val_Data_2019_512"

input_path_predict_img = "../input/isic-2019-test-input-512/ISIC_2019_Test_Input_512"


# Height and width of the input images
H = 512
W = 512

path_train_gt = "../input/train5gt/train_5_folds.csv"
path_val_gt = "../input/melanoma-detection-data/groundtruth_val.csv"

path_predict_gt = "../input/test-gt-empty-sorted/test_gt_empty_sorted.csv"


num_epochs = 9

batch_size = 8

patience_for_early_stopping = 5

learning_rate = 0.0001

## Create a Dataset class
The Dataloader needs a Pytorch Dataset format as an input. This Dataset format is created here. As input you take the gt_path and image_path

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from skimage import io, transform
import numpy as np
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from wtfml.utils import EarlyStopping


# Load Dataset
class Melanoma_Dataset(Dataset):
    def __init__(self,gt_path, image_path, transform = None, val = False):
        
        self.groundtruth = pd.read_csv(gt_path)
        self.image_path = image_path
        self.transform = transform
        
        self.composed = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
    def __len__(self):
        return len(self.groundtruth)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = os.path.join(self.image_path,
                               self.groundtruth.iloc[idx,0])
        image = io.imread(img_name+".jpg")
        # labels = self.groundtruth.iloc[idx, 1:10]
        labels = torch.tensor(self.groundtruth.iloc[idx, 1:10].astype(int))
        # labels = np.array([labels])
        
        
        if self.transform:
            image = self.transform(image)
            
        
        return image, labels

## CNN

### Explanation of the CNN Architecture
**ImageHeightxImageWidth** (Input to EfficientNet) --> **2560,10,10** (Output of EfficientNet) -(GlobalAvgPooling)-> **2560** (Input for LinearFullyConnected Layer) -> **9** (Output of the Net)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet
from torch.nn import functional as F
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics
import pretrainedmodels

class SEResnext50_32x4d(nn.Module):
    def __init__(self, pretrained= None):
        super(SEResnext50_32x4d, self).__init__()
        
        self.base_model = pretrainedmodels.__dict__[
            "se_resnext50_32x4d"
        ](pretrained=None)
        if pretrained is not None:
            self.base_model.load_state_dict(
                torch.load(
                    "../input/pretrained-model-weights-pytorch/se_resnext50_32x4d-a260b3a4.pth"
                )
            )

        self.dense_output = nn.Linear(2048, 9)
    
    def forward(self, image):
        batch_size, _, _, _ = image.shape
        
        x = self.base_model.features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)

        x = self.dense_output(x)

        return x

def GlobalAveragePooling(x):
    return x.mean(axis=-1).mean(axis=-1)

class MelanomaNet(nn.Module):
    def __init__(self, pretrained = None):
        super(MelanomaNet, self).__init__()
        if pretrained is None:
            self.efn = EfficientNet.from_pretrained("efficientnet-b7")
        else:
            self.efn = EfficientNet.from_name("efficientnet-b7")
            self.efn.load_state_dict(torch.load(pretrained), strict = False)
        
        # Classifier
        self.avgpool = GlobalAveragePooling
        self.dense_output = nn.Linear(2560, 9) # length of output vector from EfficientNetb7 is 2560
        # self.fc2a = nn.Linear(500, 9)
        
                
    def forward(self, x):
        x = x.view(-1, 3, H , W) # has to be done since EfficientNet is taking such an input
        x = self.efn.extract_features(x)
        x = self.avgpool(x)
        x = self.dense_output(x)
        return x

## Defining the training and validation function

In [ ]:
def train(model, device, train_loader, optimizer, loss_function, epoch):
    
    # Set to training mode
    model.train()
    
    # Loop over all examples
    for batch_idx, (data, target) in enumerate(train_loader):

        # Push to GPU
        data, target = data.to(device), target.to(device)

        #Reset gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):

            #Calculate outputs
            output = model(data)


            # print("output:", output)


            # print("torch.max(target,1)[1]:", torch.max(target, 1)[1])

            # Calculate loss (is this correct??)
            loss = loss_function(output, torch.max(target, 1)[1])

            # Backpropagate loss
            loss.backward()

            # Apply gradients
            optimizer.step()


        # printout every 50 batches
        if batch_idx % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{n_total_steps}], Loss: {loss.item():.4f}')
             
            
            
def val(model, device, test_loader, loss_function, prediction_return=False):
    # Set to evaluation mode
    model.eval()
    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            curr_loss = loss_function(output,torch.max(target, 1)[1])
            output = output.cpu().numpy()
            if i==0:
                predictions = output
                
                targets = target.data.cpu().numpy()
                
                loss = np.array([curr_loss.data.cpu().numpy()])
            else:
                predictions = np.concatenate((predictions,output))
                targets = np.concatenate((targets,target.data.cpu().numpy()))
                loss = np.concatenate((loss,np.array([curr_loss.data.cpu().numpy()])))
    # One-hot to index:
    
    predictions_values = predictions
    
    predictions = np.argmax(predictions, 1)
    
    targets = np.argmax(targets, 1)
    
    # Caluclate metrics
    accuracy = np.mean(np.equal(predictions,targets))
    conf_mat = confusion_matrix(targets,predictions)
    
    sensitivity = conf_mat.diagonal()/conf_mat.sum(axis=1) # taking the mean calculates the mean sensitivity over ALL classes
    
    try:
        sensitivity_mel = conf_mat[0][0]/conf_mat[0].sum()
    except:
        sensitivity_mel = "No Value"
    
    # Print metrics
    # print(classification_report(targets, predictions, digits=3))
    print("Test Accuracy:",accuracy,"Test Sensitivity (Overall):",np.mean(sensitivity), "MEL Sensitivity:", sensitivity_mel, "Test loss:",np.mean(loss))
    
    if prediction_return == True:
        print("prediction_return = True")
    
    return predictions, predictions_values, targets



def predict(model, device, test_loader, loss_function, prediction_return=False):
    # Set to evaluation mode
    model.eval()
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data= data.to(device)
            output = model(data)
            # curr_loss = loss_function(output,torch.max(target, 1)[1])
            output = output.cpu().numpy()
            if i==0:
                predictions = output
                
            else:
                predictions = np.concatenate((predictions,output))
    
    # predictions = np.argmax(predictions, 1)
    
    
    return predictions

## Creating the Actual Datasets for the Dataloader

RandomHorizontalFlip(Can be a PIL Image or torch Tensor), RandomVerticalFlip(Can be a PIL Image or torch Tensor), RandomRotation(Can be a PIL Image or torch Tensor),
transforms.ColorJitter(Can be a PIL Image or torch Tensor)


In [ ]:

mean = [0.485, 0.456, 0.406]

std = [0.229, 0.224, 0.255]



# Here we determine what transformation/augmentations we want to have on our image (e.g. flipping, rotating etc.)
train_transform = transforms.Compose([transforms.ToPILImage(),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomApply([transforms.RandomRotation(180), transforms.ColorJitter()], p =0.4),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.255))
                                     ])


val_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.255))])





# you have to take the path to "Test_Data_2019_384" - I misspelled it
train_dataset = Melanoma_Dataset(gt_path = path_train_gt,
                                image_path = input_path_training_img,
                                transform = train_transform)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size,
                                           shuffle = True)


# you have to take the path to "Test_Data_2019_384" - I misspelled it
val_dataset = Melanoma_Dataset(gt_path = path_val_gt,
                                image_path = input_path_val_img,
                                transform = val_transform, val = False)



val_loader = torch.utils.data.DataLoader(dataset = val_dataset, batch_size = batch_size,
                                           shuffle = False)


predict_dataset = Melanoma_Dataset(gt_path = path_predict_gt,
                                  image_path = input_path_predict_img,
                                  transform = val_transform, val = False)

predict_loader = torch.utils.data.DataLoader(dataset = predict_dataset, batch_size = batch_size,
                                           shuffle = False)

## Preperation for the actual training

In [ ]:
import torch.optim as optim

#GPU Info
print('Numbers of GPUs: ', torch.cuda.device_count())
print('Type of GPUs: ', torch.cuda.get_device_name(device=None))

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Model
# model = MelanomaNet(pretrained = None).to(device)

model = SEResnext50_32x4d()

#model_path = "../input/model-submission-8/model_8.bin"
#model.load_state_dict(torch.load(model_path))
#model.to(device)



# Loss
loss_function = nn.CrossEntropyLoss() #should only be used for 1 class problems??
# loss_function = nn.NLLLoss()

# Optimizer / we could implement an adaptive learning rate
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# Learning Rate scheduler
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, mode="max") #we schedule on the auc of roc curve 

# Early Stopping Library
early_stopping = EarlyStopping(patience_for_early_stopping, mode="max")



## Where we start training and validate

In [ ]:
n_total_steps = len(train_loader)

prediction_return = True

print(n_total_steps)

for epoch in range(num_epochs):
    train(model, device, train_loader, optimizer, loss_function, epoch)
        
    predictions, predictions_values, targets = val(model, device, val_loader, loss_function)
    
    ################
    # Normalize predictions_values for auc
    # Das hier ausgliedern in neue Funktion!
    pred_val_del = np.delete(predictions_values, -1, 1) # we have to delete the last entry of all predictions since the index 8 is never appearing in the targets file
    
    pred_val_torch = torch.from_numpy(pred_val_del)
    pred_val_sig = torch.sigmoid(pred_val_torch)
    row_sums  = torch.sum(pred_val_sig,1)
    row_sums_full = torch.repeat_interleave(row_sums,8)
    divider = torch.reshape(row_sums_full, (len(predictions_values),8))
    y_pred = torch.div( pred_val_sig , divider )
    y_pred_np = y_pred.numpy()

    ################
    
    auc = metrics.roc_auc_score(targets, y_pred_np, multi_class = "ovr")
    
    print(f"Epoch = {epoch}, AUC = {auc}")
    
    lr_scheduler.step(auc)
    
    early_stopping(auc, model, model_path=f"model_{epoch}.bin")
    if early_stopping.early_stop:
        print("Early stopping")
        break
    

# Load and use a saved model for prediction

In [ ]:
def predict_SEResnext50():
    model_path = "../input/model-submission-8/model_8.bin"
    
    model_pred = SEResnext50_32x4d(pretrained=None)
    model_pred.load_state_dict(torch.load(model_path))
    model_pred.to(device)
    
    new_predictions = predict(model_pred, device, val_loader, loss_function)
    # new_predictions = val(model_pred, device, val_loader, loss_function)
    
    return new_predictions
    
    
def predict_EfficientNet():
    
    model_path = "../input/model-7/model_7.bin"
    
    model_pred = MelanomaNet(pretrained = model_path)
    model_pred.to(device)
    
    new_predictions, _, _ = val(model_pred, device, val_loader, loss_function)
    
    return new_predictions


new_predictions = predict_SEResnext50()
# new_predictions = predict_EfficientNet()

In [ ]:
print(type(new_predictions))
print(len(new_predictions))
print(new_predictions)
prediction_values = new_predictions

# Create the submission csv

In [ ]:
# USE new_predictions variable from code block above to create the csv from the predictions made from an uploaded model!!!
t_p_a = new_predictions

output_df = pd.read_csv("../input/test-gt-empty-sorted/test_gt_empty_sorted.csv") # VERY IMPORTANT PUT THE GROUND TRUTH CSV FILE PATH WHICH YOU USE FOR THE DATALOADER IN HERE!!!!
image_names = output_df.drop(columns=['MEL', "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC", "UNK"])


pred_df = pd.DataFrame(columns=['MEL', "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC", "UNK"])


for i in range(len(t_p_a)):

    pred_image = np.array([0,0,0,0,0,0,0,0,0])
    np.put(pred_image, t_p_a[i],1)
    pred_df = pred_df.append({"MEL": pred_image[0], "NV": pred_image[1], "BCC": pred_image[2],
                              "AK": pred_image[3], "BKL": pred_image[4], "DF": pred_image[5],
                              "VASC": pred_image[6], "SCC": pred_image[7], "UNK": pred_image[8]},
                             ignore_index=True
                              )    

pred_df.insert(loc=0, column="image", value=image_names)

submission = pred_df

submission.to_csv("phase2_submission_group_4.csv", index=False) # Put the name to the output path as the first parameter


In [ ]:
import pandas as pd

image_names = ["image"]

dataframe = pd.DataFrame(columns=['MEL', "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC", "UNK"])

dataframe.insert(loc=0, column="image", value=image_names)

dataframe = dataframe.fillna(0)

dataframe.to_csv("single_image_predict.csv", index=False)

# Print the ROC Curve

### how they did it in the tutorial (https://scikit-learn.org/0.15/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve)

**Ablauf:**

    1. Lösche aus predictions_values den letzten Eintrag (UNK, 9ter Eintrag, [8]) --> del_pred
    2. Bringe predictions_values auf die Form (0.6,0.04,0.02,0.18,0.01,0.3,0.01,0.011) alle values zusammen müssen 1 ergeben!
        a. Das ist unser y_score
    3. Nutze die gt_val data und bringe sie auf die Form: np.array (1,0,0,0,0,0,0,0)
        b. Das ist unser y_test
        
    4. n_classes = 8 (UNK wird rausgekickt)
    
    5. Code wie ein Block weiter oben bzw. wie auf der sklearn Seite ausführen

## argmax to one hot encoded

In [ ]:
import scipy
pred = np.delete(prediction_values, -1, 1) 
print(pred[0])

pred_prob = scipy.special.softmax(pred, axis = 1)
print(pred_prob[0])
pred_prob.sum(axis= 1)

In [ ]:
del_pred = np.delete(predictions_values, -1, 1)  
del_pred_max = np.argmax(del_pred, axis = 1)


del_pred_one_hot_enc = np.zeros((del_pred_max.size, del_pred_max.max()+1))
del_pred_one_hot_enc[np.arange(del_pred_max.size),del_pred_max] = 1

y_score = del_pred_one_hot_enc

## gt_val data from csv to numpy one hot encoded

In [ ]:
gt_df = pd.read_csv("../input/melanoma-detection-data/groundtruth_val.csv")

del gt_df["image"]

gt_np = gt_df.to_numpy()

y_test = gt_np

y_test[:, 3]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

classes = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"]

plt.rcParams["figure.figsize"] = (10,10)

gt_df = pd.read_csv("../input/melanoma-detection-data/groundtruth_val.csv")
del gt_df["image"]
gt_np = gt_df.to_numpy()
y_test = gt_np
y_test[:, 3]

y_score = pred


fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

for i in range(len(classes)):
    plt.plot(fpr[i], tpr[i], label="ROC curve of class " + classes[i] + "(area = {1:0.2f})"
                                   ''.format(i, roc_auc[i]))
    
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()
